# Aritificial intelligence assignment 2024


In dit document wordt een model getraind op de mnist nummers dataset. Het model bevat 3 lagen (Flatten, Dense, Dense/output) 
waar nog mee geexperimenteerd is. Daarnaast wordt de standaard accuracy getoond voor het meten van de nauwkeurigheid van het model.
De loss function sparse_categorical_crossentropy wordt gebruikt en de nauwkeurigheid van het model is het hoogst met epochs = 5 en 
validation_split=0.1. Dit betekend dat de dataset 5x door het model heen gehaald is met forward- en backward propogation.
De validation_split van 0.1 betekend dat 1/100e van de trainingsset als validatie data/held out data dient.

In [34]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SeparableConv2D, BatchNormalization

mnist = tf.keras.datasets.mnist

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.10.0


In [35]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [36]:
# Normalize the pixel values to be between 0 and 1
# x_train = tf.keras.utils.normalize(x_train, axis=1)
# x_test = tf.keras.utils.normalize(x_test, axis=1)
x_train = (x_train - 0.0) / (255.0 - 0.0)
x_test = (x_test - 0.0) / (255.0 - 0.0)

In [37]:
# Reshape the data if using convolutional neural networks (CNNs)
# x_train = x_train.reshape(-1, 28, 28, 1)
# x_test = x_test.reshape(-1, 28, 28, 1)
x_train = x_train.reshape((x_train.shape + (1,)))
x_test = x_test.reshape((x_test.shape + (1,)))

In [38]:
# Toon de data
print("Training data shape:", x_train.shape)
print("Test data shape:", x_test.shape)

Training data shape: (60000, 28, 28, 1)
Test data shape: (10000, 28, 28, 1)


In [39]:
model = Sequential()

#Maak de 28x28 array plat tot een 1 dimentionale array/ of 
# model.add(Flatten(input_shape=(28, 28)))

#Check wat efficienter is voor de nauwkeurigheid van het model, Conv2D or double Dense layers
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(10, activation='softmax'))

#Compile the current model
optimizer = SGD(learning_rate=0.01, momentum=0.9)

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer,
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

Na zorgvuldig testen is de conclusie dat er tussen de SDG optimizer en de adam optimizer weinig verschil zit in totale performance.
De sdg optimizer gaat bij de eerste epoch trager, maar bij de tweede epoch sneller dan de adam optimizer wat in totaal uitkomt op hetzelfde.

In [40]:
# Train het model met validatie
history = model.fit(x_train, y_train, epochs=2, batch_size=32)

Epoch 1/2
1875/1875 [==============================] - 71s 38ms/step - loss: 0.1863 - accuracy: 0.9420
Epoch 2/2
1875/1875 [==============================] - 69s 37ms/step - loss: 0.0524 - accuracy: 0.9839


In [41]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1600)              0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [42]:
# Evalueer het model op de testset
test_loss, test_accuracy = model_for_pruning.evaluate(x_test, y_test)

print(f"Test nauwkeurigheid: {test_accuracy:.4f}")


313/313 [==============================] - 2s 5ms/step - loss: 2.3082 - accuracy: 0.0939
Test nauwkeurigheid: 0.0939


In [43]:
# Modelgrootte berekenen
model_size = model.count_params()
model_memory = model_size * 4 / (1024 ** 2)  # 4 bytes per 32-bit float
print(f"Modelgrootte: {model_memory:.2f} MB")

Modelgrootte: 0.93 MB


# In dit deel wordt een nauwkeurigheid van het model getoond door middel van de confusion matrix

In [44]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [45]:
# Maak voorspellingen op de testset
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

313/313 [==============================] - 2s 4ms/step


In [46]:
# Bereken de verwarringsmatrix
cm = confusion_matrix(y_test, y_pred_classes)
print("Verwarringsmatrix:")
print(cm)

Verwarringsmatrix:
[[ 970    0    1    0    0    0    1    3    5    0]
 [   0 1110    2    4    0    0    5    3   11    0]
 [   1    0 1026    0    0    0    0    3    2    0]
 [   0    0    3 1003    0    0    0    2    2    0]
 [   0    0    3    1  953    0    4    0    4   17]
 [   1    0    0   24    0  857    1    1    2    6]
 [   5    2    0    0    1    4  945    0    1    0]
 [   0    0    7    6    0    0    0 1010    1    4]
 [   2    0    3    1    0    1    0    3  963    1]
 [   2    2    0    5    1    0    0    5    3  991]]


De verwarringsmatrix is te lezen als volgt:
De x as moeten de getallen van 0 tm 9 bevatten die het model voorspelt.
De y as moeten de getallen zijn die gelabeld zijn als 0 tm 9.

In deze matrix kunnen we zien dat klasse 8 op de y-as het meest fout gedaan wordt door het model. Daarnaast kan hieruit geconcludeerd worden dat klasse 7 op de x as het meest fout geraden is door het model